# MLP with SGD Using Tensorflow - Tuning

In [9]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import wandb
from pprint import pprint

from tf_utils import create_model, train_model

## Login to Wandb

In [10]:
wandb.login()

True

## Load Fashion-MNIST

In [11]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


## Define sweep agent function

In [12]:
def sweep_train():
    default_config = {
        'dense_1': 512,
        'dense_2': 256,
        'dense_3': 256,
        'dense_4': 64,
        'learning_rate': 0.001,
        'momentum': 0.0,
        'epochs': 50,
        'batch_size': 128,
    }

    wandb.init(config=default_config)

    config = wandb.config

    mlp = create_model(config)
    mlp.summary()
    mlp.compile(
        optimizer=SGD(learning_rate=config.learning_rate, momentum=config.momentum),
        loss=SparseCategoricalCrossentropy(),
        metrics=[SparseCategoricalAccuracy()]
    )
    _ = train_model(mlp, config, x_train, y_train, verbose=0)

## Configure Wandb Sweep for hyperparameter tuning

In [13]:
sweep_config = {
    'name': 'tf-sgd-sweep',
    'method': 'bayes',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'early_terminate':{
        'type': 'hyperband',
        'min_iter': 2
    },
    'parameters': {
        'batch_size': {
            'values': [16, 32, 64, 128, 256]
        },
        'epochs': {
            'min': 15,
            'max': 50
        },
        'learning_rate': {
            'min': 0.0001,
            'max': 0.1
        },
        'momentum': {
            'min': 0.0,
            'max': 0.9
        },
        'dense_1': {
            'min': 256,
            'max': 512
        },
        'dense_2': {
            'min': 128,
            'max': 256
        },
        'dense_3': {
            'min': 128,
            'max': 256
        },
        'dense_4': {
            'min': 32,
            'max': 128
        }
    }
}

pprint(sweep_config)

sweep_id = wandb.sweep(sweep_config, project='tf_mlp')

{'early_terminate': {'min_iter': 2, 'type': 'hyperband'},
 'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'name': 'tf-sgd-sweep',
 'parameters': {'batch_size': {'values': [16, 32, 64, 128, 256]},
                'dense_1': {'max': 512, 'min': 256},
                'dense_2': {'max': 256, 'min': 128},
                'dense_3': {'max': 256, 'min': 128},
                'dense_4': {'max': 128, 'min': 32},
                'epochs': {'max': 50, 'min': 15},
                'learning_rate': {'max': 0.1, 'min': 0.0001},
                'momentum': {'max': 0.9, 'min': 0.0}}}
Create sweep with ID: ifkgs608
Sweep URL: https://wandb.ai/nsiete23/tf_mlp/sweeps/ifkgs608


## Run sweep

In [14]:
wandb.agent(sweep_id, function=sweep_train, count=50)

wandb: Agent Starting Run: uodavz95 with config:
wandb: 	batch_size: 16
wandb: 	dense_1: 261
wandb: 	dense_2: 202
wandb: 	dense_3: 168
wandb: 	dense_4: 70
wandb: 	epochs: 41
wandb: 	learning_rate: 0.01336609504054038
wandb: 	momentum: 0.34330504899734904


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 28, 28)            0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 261)               204885    
                                                                 
 dense_1 (Dense)             (None, 202)               52924     
                                                                 
 dense_2 (Dense)             (None, 168)               34104     
                                                                 
 dense_3 (Dense)             (None, 70)                11830     
                                                                 
 dense_4 (Dense)             (None, 10)                7

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
loss,█▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
sparse_categorical_accuracy,▁▃▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
val_loss,▇▆▄▃▂▂▃▂▂▁▂▃▂▁▃▃▄▂▃▂▃▃▄▃▄▄▃▄▄▅▅▅▆▆▅▅▅▆█▆
val_sparse_categorical_accuracy,▁▂▄▅▅▅▆▆▇▇▆▆▇█▆▆▆███▇▆▇▆▇▇▇█▇▇▇█▇▇▇██▇██
GFLOPS,0.0003
best_epoch,13
best_val_loss,0.30114
epoch,40
loss,0.09645
sparse_categorical_accuracy,0.96288


wandb: Agent Starting Run: 148nbd56 with config:
wandb: 	batch_size: 128
wandb: 	dense_1: 278
wandb: 	dense_2: 235
wandb: 	dense_3: 184
wandb: 	dense_4: 46
wandb: 	epochs: 30
wandb: 	learning_rate: 0.08979136374875403
wandb: 	momentum: 0.5220533768294469


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 28, 28)            0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 278)               218230    
                                                                 
 dense_1 (Dense)             (None, 235)               65565     
                                                                 
 dense_2 (Dense)             (None, 184)               43424     
                                                                 
 dense_3 (Dense)             (None, 46)                8510      
                                                                 
 dense_4 (Dense)             (None, 10)                4

...

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 28, 28)            0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 452)               354820    
                                                                 
 dense_1 (Dense)             (None, 240)               108720    
                                                                 
 dense_2 (Dense)             (None, 136)               32776     
                                                                 
 dense_3 (Dense)             (None, 81)                11097     
                                                                 
 dense_4 (Dense)             (None, 10)                8

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▄▃▃▂▂▂▂▂▁▁▁▁▁▁
sparse_categorical_accuracy,▁▅▆▆▇▇▇▇▇██████
val_loss,█▄▄▃▅▂▂▃▁▂▅▁▁▂▂
val_sparse_categorical_accuracy,▁▅▅▆▃▆█▆█▇▅██▇▇
GFLOPS,0.00051
best_epoch,12
best_val_loss,0.33738
epoch,14
loss,0.27505
sparse_categorical_accuracy,0.89658


wandb: Agent Starting Run: 1wswtflf with config:
wandb: 	batch_size: 256
wandb: 	dense_1: 347
wandb: 	dense_2: 196
wandb: 	dense_3: 149
wandb: 	dense_4: 75
wandb: 	epochs: 15
wandb: 	learning_rate: 0.05911317122278366
wandb: 	momentum: 0.07199184672184085


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 28, 28)            0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 347)               272395    
                                                                 
 dense_1 (Dense)             (None, 196)               68208     
                                                                 
 dense_2 (Dense)             (None, 149)               29353     
                                                                 
 dense_3 (Dense)             (None, 75)                11250     
                                                                 
 dense_4 (Dense)             (None, 10)                7

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▄▃▃▂▂▂▂▂▂▁▁▁▁▁
sparse_categorical_accuracy,▁▅▆▆▇▇▇▇▇▇█████
val_loss,█▄▃▃▂▃▂▂▂▂▂▁▁▂▁
val_sparse_categorical_accuracy,▁▆▆▇▇▇▇▇▇▇▇██▇█
GFLOPS,0.00038
best_epoch,11
best_val_loss,0.34854
epoch,14
loss,0.28843
sparse_categorical_accuracy,0.89402


## Show best hyperparameters

In [15]:
api = wandb.Api()
sweep = api.sweep("nsiete23/tf_mlp/sweeps/" + sweep_id)

best_run = sweep.best_run()
print(best_run.id)
pprint(best_run.config)

wandb: Sorting runs by +summary_metrics.val_loss


r3fkcitd
{'batch_size': 128,
 'dense_1': 414,
 'dense_2': 205,
 'dense_3': 141,
 'dense_4': 106,
 'epochs': 20,
 'learning_rate': 0.08418417163921542,
 'momentum': 0.31453202807460157}


## Finish Wandb run

In [16]:
wandb.finish()